In [ ]:
pl_data = []
laliga_data = []
seria_data = []
bundes_data = []
ligue1_data = []

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
headers = {
    'User-Agent': ''}

def get_team_results(team, team_url):
    response = requests.get(team_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    matches = soup.find_all('tr', class_='Table__TR Table__TR--sm Table__even')
    match_results = []
    for match in matches:
        date = match.find('div', class_='matchTeams').text if match.find('div', class_='matchTeams') else ''
        opponents=[]
        count = 0
        for opponent in match:
            opponent = match.find_all('a', class_='AnchorLink Table__Team')
            count = 0
            home = True
            for link in opponent:
                if team not in link.text:
                    opponents.append(link.text)
                    break
                count += 1
        if len(opponents) == 0:
            op = team
        else:
            op = opponents[0]
        if count == 0:
            home = False
            count = 0

        scores = []
        for result in match:
            result = match.find_all('a', class_='AnchorLink', href=True)
            for link in result:
                if ' - ' in link.text:
                    scores.append(link.text)
        res = scores[0]
        win = eval(res)
        r = '.'
        if home:
            if win > 0:
                r = 'Win'
            elif win == 0:
                r = 'Draw'
            else:
                r = 'Lose'
        else:
            if win < 0:
                r = 'Win'
            elif win == 0:
                r = 'Draw'
            else:
                r = 'Lose'

        competitions = []
        for comp in match:
            comp = match.find_all('td', class_='Table__TD')
            for link in comp:
                if 'English Premier League' in link.text:
                    competitions.append(link.text)
                elif 'Spanish LALIGA' in link.text:
                    competitions.append(link.text)
                elif 'Italian Serie A' in link.text:
                    competitions.append(link.text)
                elif 'German Bundesliga' in link.text:
                    competitions.append(link.text)
                elif 'French Ligue 1' in link.text:
                    competitions.append(link.text)

        if len(competitions) == 0:
            competitions.append('.')

        competition = competitions[0]

        h = 'Home'
        if not home:
            h = 'Away'

        # 홈 경기의 경우 eval(res)를 그대로 사용
        if home:
            if r == 'Win':
                result_ = eval(res)
            elif r == 'Draw':
                result_ = 0
            else:
                result_ =  eval(res)
        # 원정 경기의 경우 eval(res)에 -1을 곱해서 사용
        else:
            if r == 'Win':
                result_ = -eval(res)
            elif r == 'Draw':
                result_ = 0
            else:
                result_ = -eval(res)

        if competition == 'English Premier League':
            pl_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'Spanish LALIGA':
            laliga_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'Italian Serie A':
            seria_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'German Bundesliga':
            bundes_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'French Ligue 1':
            ligue1_data.append({'Team': team, 'Opponent': op, 'Result': result_})


        match_results.append({
            'Date': date,
            'Opponent': op,
            'Score': '\'' + res,
            'Competition': competition,
            'Result': r,
            'Home': h
        })

    return match_results

team_urls = {
    # EPL
    'Arsenal': 'https://www.espn.com/soccer/team/results/_/id/359/season/2022',
    'Aston Villa': 'https://www.espn.com/soccer/team/results/_/id/362/season/2022',
    'Bournemouth': 'https://www.espn.com/soccer/team/results/_/id/349/season/2022',
    'Brentford': 'https://www.espn.com/soccer/team/results/_/id/337/season/2022',
    'Brighton & Hove Albion': 'https://www.espn.com/soccer/team/results/_/id/331/season/2022',
    'Chelsea': 'https://www.espn.com/soccer/team/results/_/id/363/season/2022',
    'Crystal Palace': 'https://www.espn.com/soccer/team/results/_/id/384/season/2022',
    'Everton': 'https://www.espn.com/soccer/team/results/_/id/368/season/2022',
    'Fulham': 'https://www.espn.com/soccer/team/results/_/id/370/season/2022',
    'Leeds United': 'https://www.espn.com/soccer/team/results/_/id/357/season/2022',
    'Leicester City': 'https://www.espn.com/soccer/team/results/_/id/375/season/2022',
    'Liverpool': 'https://www.espn.com/soccer/team/results/_/id/364/season/2022',
    'Manchester City': 'https://www.espn.com/soccer/team/results/_/id/382/season/2022',
    'Manchester United': 'https://www.espn.com/soccer/team/results/_/id/360/season/2022',
    'Newcastle United': 'https://www.espn.com/soccer/team/results/_/id/361/season/2022',
    'Nottingham Forest': 'https://www.espn.com/soccer/team/results/_/id/393/season/2022',
    'Southampton': 'https://www.espn.com/soccer/team/results/_/id/376/season/2022',
    'Tottenham Hotspur': 'https://www.espn.com/soccer/team/results/_/id/367/season/2022',
    'West Ham United': 'https://www.espn.com/soccer/team/results/_/id/371/season/2022',
    'Wolverhampton Wanderers': 'https://www.espn.com/soccer/team/results/_/id/380/season/2022',

    # LALIGA
    'Almería': 'https://www.espn.com/soccer/team/results/_/id/6832/season/2022',
    'Athletic Club': 'https://www.espn.com/soccer/team/results/_/id/93/season/2022',
    'Atlético Madrid': 'https://www.espn.com/soccer/team/results/_/id/1068/season/2022',
    'Barcelona': 'https://www.espn.com/soccer/team/results/_/id/83/season/2022',
    'Cádiz': 'https://www.espn.com/soccer/team/results/_/id/3842/season/2022',
    'Celta Vigo': 'https://www.espn.com/soccer/team/results/_/id/85/season/2022',
    'Elche': 'https://www.espn.com/soccer/team/results/_/id/3751/season/2022',
    'Espanyol': 'https://www.espn.com/soccer/team/results/_/id/88/season/2022',
    'Getafe': 'https://www.espn.com/soccer/team/results/_/id/2922/season/2022',
    'Girona': 'https://www.espn.com/soccer/team/results/_/id/9812/season/2022',
    'Mallorca': 'https://www.espn.com/soccer/team/results/_/id/84/season/2022',
    'Osasuna': 'https://www.espn.com/soccer/team/results/_/id/97/season/2022',
    'Rayo Vallecano': 'https://www.espn.com/soccer/team/results/_/id/101/season/2022',
    'Real Betis': 'https://www.espn.com/soccer/team/results/_/id/244/season/2022',
    'Real Madrid': 'https://www.espn.com/soccer/team/results/_/id/86/season/2022',
    'Real Sociedad': 'https://www.espn.com/soccer/team/results/_/id/89/season/2022',
    'Sevilla': 'https://www.espn.com/soccer/team/results/_/id/243/season/2022',
    'Valencia': 'https://www.espn.com/soccer/team/results/_/id/94/season/2022',
    'Valladolid': 'https://www.espn.com/soccer/team/results/_/id/95/season/2022',
    'Villarreal': 'https://www.espn.com/soccer/team/results/_/id/102/season/2022',

    # SERIA A
    'AC Milan': 'https://www.espn.com/soccer/team/results/_/id/103/season/2022',
    'AS Roma': 'https://www.espn.com/soccer/team/results/_/id/104/season/2022',
    'Atalanta': 'https://www.espn.com/soccer/team/results/_/id/105/season/2022',
    'Bologna': 'https://www.espn.com/soccer/team/results/_/id/107/season/2022',
    'Cremonese': 'https://www.espn.com/soccer/team/results/_/id/4050/season/2022',
    'Empoli': 'https://www.espn.com/soccer/team/results/_/id/2574/season/2022',
    'Fiorentina': 'https://www.espn.com/soccer/team/results/_/id/109/season/2022',
    'Hellas Verona': 'https://www.espn.com/soccer/team/results/_/id/119/season/2022',
    'Internazionale': 'https://www.espn.com/soccer/team/results/_/id/110/season/2022',
    'Juventus': 'https://www.espn.com/soccer/team/results/_/id/111/season/2022',
    'Lazio': 'https://www.espn.com/soccer/team/results/_/id/112/season/2022',
    'Lecce': 'https://www.espn.com/soccer/team/results/_/id/113/season/2022',
    'Monza': 'https://www.espn.com/soccer/team/results/_/id/4007/season/2022',
    'Napoli': 'https://www.espn.com/soccer/team/results/_/id/114/season/2022',
    'Salernitana': 'https://www.espn.com/soccer/team/results/_/id/3240/season/2022',
    'Sampdoria': 'https://www.espn.com/soccer/team/results/_/id/2734/season/2022',
    'Sassuolo': 'https://www.espn.com/soccer/team/results/_/id/3997/season/2022',
    'Spezia': 'https://www.espn.com/soccer/team/results/_/id/4056/season/2022',
    'Torino': 'https://www.espn.com/soccer/team/results/_/id/239/season/2022',
    'Udinese': 'https://www.espn.com/soccer/team/results/_/id/118/season/2022',

    # BUNDESLIGEA
    'FC Augsburg': 'https://www.espn.com/soccer/team/results/_/id/3841/season/2022',
    'Hertha Berlin': 'https://www.espn.com/soccer/team/results/_/id/129/season/2022',
    '1. FC Union Berlin': 'https://www.espn.com/soccer/team/results/_/id/598/season/2022',
    'Werder Bremen': 'https://www.espn.com/soccer/team/results/_/id/137/season/2022',
    'Borussia Dortmund': 'https://www.espn.com/soccer/team/results/_/id/124/season/2022',
    'Eintracht Frankfurt': 'https://www.espn.com/soccer/team/results/_/id/125/season/2022',
    'SC Freiburg': 'https://www.espn.com/soccer/team/results/_/id/126/season/2022',
    'TSG Hoffenheim': 'https://www.espn.com/soccer/team/results/_/id/7911/season/2022',
    'FC Cologne': 'https://www.espn.com/soccer/team/results/_/id/122/season/2022',
    'RB Leipzig': 'https://www.espn.com/soccer/team/results/_/id/11420/season/2022',
    'Bayer Leverkusen': 'https://www.espn.com/soccer/team/results/_/id/131/season/2022',
    'Mainz': 'https://www.espn.com/soccer/team/results/_/id/2950/season/2022',
    'Borussia Monchengladbach': 'https://www.espn.com/soccer/team/results/_/id/268/season/2022',
    'Bayern Munich': 'https://www.espn.com/soccer/team/results/_/id/132/season/2022',
    'VfL Bochum': 'https://www.espn.com/soccer/team/results/_/id/121/season/2022',
    'VfB Stuttgart': 'https://www.espn.com/soccer/team/results/_/id/134/season/2022',
    'VfL Wolfsburg': 'https://www.espn.com/soccer/team/results/_/id/138/season/2022',
    'Schalke 04': 'https://www.espn.com/soccer/team/results/_/id/133/season/2022',

    # LIGUE 1
    'AC Ajaccio': 'https://www.espn.com/soccer/team/results/_/id/2503/season/2022',
    'Angers': 'https://www.espn.com/soccer/team/results/_/id/7868/season/2022',
    'AJ Auxerre': 'https://www.espn.com/soccer/team/results/_/id/172/season/2022',
    'AS Monaco': 'https://www.espn.com/soccer/team/results/_/id/174/season/2022',
    'Clermont Foot': 'https://www.espn.com/soccer/team/results/_/id/3171/season/2022',
    'Troyes': 'https://www.espn.com/soccer/team/results/_/id/170/season/2022',
    'Lorient': 'https://www.espn.com/soccer/team/results/_/id/273/season/2022',
    'Lille': 'https://www.espn.com/soccer/team/results/_/id/166/season/2022',
    'Montpellier': 'https://www.espn.com/soccer/team/results/_/id/274/season/2022',
    'Nantes': 'https://www.espn.com/soccer/team/results/_/id/165/season/2022',
    'Nice': 'https://www.espn.com/soccer/team/results/_/id/2502/season/2022',
    'Lyon': 'https://www.espn.com/soccer/team/results/_/id/167/season/2022',
    'Marseille': 'https://www.espn.com/soccer/team/results/_/id/176/season/2022',
    'Paris Saint-Germain': 'https://www.espn.com/soccer/team/results/_/id/160/season/2022',
    'Lens': 'https://www.espn.com/soccer/team/results/_/id/175/season/2022',
    'Brest': 'https://www.espn.com/soccer/team/results/_/id/6997/season/2022',
    'Stade de Reims': 'https://www.espn.com/soccer/team/results/_/id/3243/season/2022',
    'Stade Rennais': 'https://www.espn.com/soccer/team/results/_/id/169/season/2022',
    'Strasbourg': 'https://www.espn.com/soccer/team/results/_/id/180/season/2022',
    'Toulouse': 'https://www.espn.com/soccer/team/results/_/id/179/season/2022'
}

all_team_results = {}

for team, url in team_urls.items():
    results = get_team_results(team, url)
    all_team_results[team] = results

for team, results in all_team_results.items():
    df = pd.DataFrame(results)
    #print(f"\n{team} Match Results:")
    #print(df)

In [ ]:
for team, results in all_team_results.items():
    df = pd.DataFrame(results)
    df = df[df['Competition'] != '.']
    file_name = f'{team}_2223.csv'
    df.to_csv(file_name, index=False, encoding = "utf-8-sig")


In [ ]:
pl_df = pd.DataFrame(pl_data)
pl_df.to_csv('pl_df_2223.csv', index=False, encoding="utf-8-sig")

laliga_df = pd.DataFrame(laliga_data)
laliga_df.to_csv('laliga_df_2223.csv', index=False, encoding="utf-8-sig")

seria_df = pd.DataFrame(seria_data)
seria_df.to_csv('seria_df_2223.csv', index=False, encoding="utf-8-sig")

bundes_df = pd.DataFrame(bundes_data)
bundes_df.to_csv('bundes_df_2223.csv', index=False, encoding="utf-8-sig")

ligue1_df = pd.DataFrame(ligue1_data)
ligue1_df.to_csv('ligue1_df_2223.csv', index=False, encoding="utf-8-sig")
